### import full-fred to query FRED data

In [35]:
try: 
  from full_fred.fred import Fred
except:
  !pip install full-fred
  from full_fred.fred import Fred

from time import sleep
from tqdm import tqdm
import pandas as pd

### set API key

In [36]:
fred = Fred('api_key')

### find the right category id of 'unemployment rate' and 'series ids' of all counties

In [37]:
# utility functions to go around FED's daily request limit
def nap(n):
    for _ in tqdm(range(n), desc = 'zzz.. napping'):
        sleep(1)

def keep_trying(func, x, y, no_subscription = False):
    error = True
    n_trial = 0
    if not no_subscription:
        while(error):
            try:
                return func(x)[y]
            except:
                nap(1 + n_trial * 1)
                n_trial += 1
    else:
        while(error):
            try:
                response = func(x)
                if type(response) == pd.DataFrame:
                    return response
                else:
                    raise Exception
            except:
                nap(1 + n_trial * 1)
                n_trial += 1


In [38]:
# define utility functions to explore the file hierachy of FRED DB
def search(id : str, up : bool, cat : bool):
    '''find the parent / children of a category / series'''
    if cat:
        if up:
            return fred.get_a_category(id)
        else:
            return fred.get_child_categories(id) 
    else:  # series
        if up:
            return fred.get_categories_of_series(id)
        else:
            return fred.get_a_series(id)

In [39]:
n = 0

with open('id', 'w+') as file:
  for state_dict in fred.get_child_categories(27281)['categories']:
    print ('\n')
    print (state_dict['name'])
    state_id = state_dict['id']
    state_item_list = keep_trying(fred.get_child_categories, state_id, 'categories')
    # fred.get_child_categories(state_id)['categories']

    no_county = True  # by default, assume there's no county in a state, like Alaska
    for state_item in state_item_list:
      if state_item['name'] == 'Counties':  # find cat id of County (rather than MSA)
        cat_id = state_item['id']
        county_list = keep_trying(fred.get_child_categories, cat_id, 'categories')
        # fred.get_child_categories(cat_id)['categories']
        no_county = False  # flag that 'Counties' is found
        break  # break the search loop

    if no_county:
      continue  # no county in the state, move on the next

    for county in county_list:
      id = county['id']
      name = county['name']

      item_list = keep_trying(fred.get_series_in_a_category, id, 'seriess')
      # fred.get_series_in_a_category(id)['seriess']

        
      for item in item_list:
        if item['title'][:17] == 'Unemployment Rate' and item['frequency'] == 'Monthly':  # find the series of 'unemployment rate'
          temp_series = item['id']
          temp_title = item['title']
          file.write(f'{temp_series}, {temp_title}\n')
          n += 1
          print(f'{temp_title} , {n} -----------------', end = '\r')
          break  # break out from the inner search loop



Alabama


zzz.. napping: 100%|██████████| 2/2 [00:02<00:00,  1.01s/it]


Unemployment Rate in Winston County, AL , 67 --------------------

Alaska


Arizona
Unemployment Rate in Yuma County, AZ , 82 -----------------------

Arkansas


zzz.. napping: 100%|██████████| 8/8 [00:08<00:00,  1.00s/it]


Unemployment Rate in Yell County, AR , 157 -------------------------

California


...

zzz.. napping: 100%|██████████| 9/9 [00:09<00:00,  1.00s/it]


Unemployment Rate in Yakima County, WA , 2955 -----------------------

Wisconsin


zzz.. napping: 100%|██████████| 8/8 [00:08<00:00,  1.01s/it]


Unemployment Rate in Wood County, WI , 3027 ------------------------

Wyoming


zzz.. napping: 100%|██████████| 8/8 [00:08<00:00,  1.00s/it]


### query and save unemployment data

In [31]:
id_name_list = []
with open('id', 'r') as file:
    for row in file:
        splitted_str = row.split(',')
        if len(splitted_str) == 3:
            id, county, state = splitted_str
            state = state[: - 1]  # remove line break
            county = county[22:]  # remove 'unemployment rate in'
            id_name_list.append([id, f'{county}_{state}'])
        else:
            id, county = splitted_str
            county = county[22:]  # remove 'unemployment rate in'
            id_name_list.append([id, f'{county}'])

In [57]:
for id, name in tqdm(id_name_list[194 + 160 + 129:]):
    temp_df = keep_trying(fred.get_series_df, id, '', no_subscription = True)
    name = name.replace('/', '_')
    temp_df.to_csv(f'fred_unemployment_rate/{name}', index = False)

  9%|▉         | 234/2567 [01:05<10:01,  3.88it/s]

Error Message: Too Many Requests.  Exceeded Rate Limit


zzz.. napping: 100%|██████████| 1/1 [00:01<00:00,  1.01s/it]


Error Message: Too Many Requests.  Exceeded Rate Limit


zzz.. napping: 100%|██████████| 2/2 [00:02<00:00,  1.01s/it]


...

zzz.. napping: 100%|██████████| 4/4 [00:04<00:00,  1.03s/it]


Error Message: Too Many Requests.  Exceeded Rate Limit


zzz.. napping: 100%|██████████| 5/5 [00:05<00:00,  1.01s/it]


Error Message: Too Many Requests.  Exceeded Rate Limit


100%|██████████| 2567/2567 [20:48<00:00,  2.06it/s]
